In [ ]:
import sys

sys.tracebacklimit = None

# Parameters & constants

In [ ]:
from pathlib import Path

import numpy as np

lx = 50  # length in x direction (Å)
ly = lx / np.sqrt(3)  # length in y direction (Å) keeping the b/a ratio
lz = 10 # length in z direction (Å)
nn = 1  # number of nearest neighbors | don't use 0!

basepath = Path(f"output/example/len_{lx}x{int(ly)}/nn_{nn}")
basepath.mkdir(parents=True, exist_ok=True)

# Define structure

In [ ]:
from tb_hamiltonian.utils import get_structure

structure = get_structure(
    unit_cell_filepath=Path("output/example/POSCAR"),  # local unit cell file
    lengths=(lx, ly, lz),
)

structure.info["label"] = "BLG"  # will show up at top of Hamiltonian output file

structure.write(basepath / "POSCAR", format="vasp")

# Compute H

In [ ]:
from tb_hamiltonian import TightBindingHamiltonian

H = TightBindingHamiltonian(
    structure=structure,
    nearest_neighbor=nn,
    distances=(0.0, 1.425, 2.468, 2.850),
    hopping_parameters=(0.0, -2.7, 0.0, -0.27),
    interlayer_coupling=0.33,
)

H.build()

# Apply onsite term

In [ ]:
from tb_hamiltonian.potentials import PotentialFactory

potential = PotentialFactory("triangular")
potential.params = {
    "amplitude": 1.0,
    "width": 1.0,
    "height": 2.0,  # twice the width if maintaining non-standard unit cell b/a ratio
}

H.update_onsite_terms(
    onsite_term=0.0,
    potential=potential,
    alpha=(1.0, 0.5),
)

path = (
    basepath
    / f"{potential.name}"
    / f"amplitude_{potential.params['amplitude']}"
    / f"width_{potential.params['width']}"
)
path.mkdir(parents=True, exist_ok=True)

# Sanity checks


In [ ]:
# H.onsite_count()

In [ ]:
# H.interaction_counts()

# Write H to file

In [ ]:
H.write_to_file(path)

# Plotting

In [ ]:
# H.plot_matrix()

In [ ]:
# H.plot_grid()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def apply_potential(params: dict, coordinates: np.ndarray) -> float:
    amplitude = params.get("amplitude", 1.0)
    width = params.get("width", 1.0)
    height = params.get("height", 1.0)
    n = params.get("n", 1)
    m = params.get("m", 1)
    x, y, _ = coordinates
    kx1 = 2 * np.pi * n / width
    ky1 = 2 * np.pi * m / height  # factor of 2 works only if b/a ratio is maintained

    return amplitude * (
        np.cos(kx1 * x) + np.cos(kx1 / 2 * x + ky1 * y) + np.cos(kx1 / 2 * x - ky1 * y)
    )


frac = structure.get_scaled_positions()
n = int(len(structure) // 2)
x_frac = frac[:, 0]
y_frac = frac[:, 1]
z_frac = frac[:, 2]
coords = np.array([x_frac, y_frac, z_frac]).T
width = 0.25  # 0.1 | 0.125 | 0.25 | 0.5 | 1.0
params = {
    "amplitude": 1.0,
    "width": width,
    "height": width * 2,
}
V = np.array([apply_potential(params, c) for c in coords])

plt.figure(figsize=(10, 6))

# plot the potential
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, y)
coords = np.array([X.flatten(), Y.flatten(), np.zeros_like(X.flatten())]).T
Z = np.array([apply_potential(params, c) for c in coords]).reshape(X.shape)
plt.contour(X, Y, Z, levels=250, cmap="rainbow", alpha=0.1, zorder=0)

# plot the bonds (slow!!!)
# distances = np.sqrt(
#     (x_frac[:n, np.newaxis] - x_frac[np.newaxis, :n]) ** 2
#     + (y_frac[:n, np.newaxis] - y_frac[np.newaxis, :n]) ** 2
# )
# indices = np.where((distances <= 0.03) & (distances > 0))
#
# for i, j in zip(indices[0], indices[1]):
#     plt.plot([x_frac[i], x_frac[j]], [y_frac[i], y_frac[j]], "k-", lw=0.5, zorder=1)

# plot the atoms
plt.scatter(x_frac[:n], y_frac[:n], s=10, c=V[:n], cmap="rainbow", zorder=2)

plt.show()